In [1]:
import pandas as pd
import os
import requests
import io
from bs4 import BeautifulSoup 
import re
import numpy as np
import zipfile as zf
from io import BytesIO
import pint
from pint import UnitRegistry
ureg=pint.UnitRegistry()

## Saving desired raw JRC datasets

In [2]:
## Saving desired raw JRC datasets

JRC_URL= 'https://zenodo.org/record/3349843#.XVLRjnspCUl'

r = requests.get(JRC_URL) 
print(r)
soup = BeautifulSoup(r.text,'html.parser')

#storing all 'a' tags
links= soup.find_all('a',attrs={'class':'filename'})

#keeping only links to zip
zip_available=[]
for item in links:
    urls= 'https://zenodo.org'+item.get('href')
    zip_available.append(urls)
    
#link to zip folder 
jrc_dat=zip_available[-1]
        
#getting a list of files in the zip folder 
r2 = requests.get(jrc_dat, stream=True)
with zf.ZipFile(BytesIO(r2.content)) as z:
    # z is a zipfile object
    file_names = z.namelist()
    
    csvs=[]
    for file in file_names:
        if file.endswith('.csv'):
            csvs.append(file)
        else:
            pass
        
    #keeping only OPEN_UNITS and OPEN_TEMPORAL
    params_remove= ['PERFORMANCE']
    jrc_datasets= [csv_file for csv_file in csvs if not any(param in csv_file for param in params_remove)]

    print(jrc_datasets)
    
    dat_names=[]
    for dataset in jrc_datasets:
        name= dataset.split('_')[2]
        file_name= name.split('.')[0]
        dat_names.append(file_name)
    
    for csv, f_name in zip(jrc_datasets,dat_names):
        new_file_name= 'raw_europe_{}.csv'.format(f_name)
        target_path= os.getcwd()+'/' + new_file_name
        with open(target_path, 'wb') as f:
            f.write(z.read(csv))

<Response [200]>
['JRC_OPEN_LINKAGES.csv', 'JRC_OPEN_TEMPORAL.csv', 'JRC_OPEN_UNITS.csv']


## Reading raw csv files

In [3]:
jrc_units = pd.read_csv('raw_europe_UNITS.csv')
jrc_co2 = pd.read_csv('raw_europe_TEMPORAL.csv')
jrc_id = pd.read_csv('raw_europe_LINKAGES.csv')

## Data Cleaning

Merging jrc_co2 and jrc_units

In [4]:
jrc_df = pd.merge(jrc_co2,jrc_units,'left',on=['eic_p','eic_g','type_g'])

Keeping operating power plants

In [5]:
jrc_df = jrc_df[jrc_df['status_g'] == 'COMMISSIONED']

Aggregating at the plant level by year

In [6]:
wm = lambda x: np.average(x,weights=jrc_df.loc[x.index,'capacity_g'])
dominant_fuel_type = lambda x: x.value_counts().index[0]

jrc_plant = jrc_df.groupby(['eic_p', 'cyear']).agg({'name_p':'first',
                                                    'capacity_p':'first',
                                                    'type_g':dominant_fuel_type,
                                                    'lat':'first',
                                                    'lon':'first',
                                                    'country':'first',
                                                    'status_g':'first',
                                                    'year_commissioned':wm,
                                                    'Generation':'sum',
                                                    'time_coverage':wm,
                                                    'co2emitted':'sum',
                                                    'eic_g':pd.Series.nunique}) #creating a column for unit_count

jrc_plant.reset_index(inplace = True)
jrc_plant.rename(columns={'eic_g':'unit_count'}, inplace=True) #naming count column "unit_count"

Adding WRI_id

In [7]:
jrc_id = jrc_id[['eic_p','WRI_id']].drop_duplicates()

In [8]:
jrc_plant = pd.merge(jrc_plant,jrc_id[['eic_p','WRI_id']],on='eic_p', how='left')

Dropping unwanted columns

In [9]:
jrc_plant.drop(['eic_p', 'status_g', 'time_coverage'], axis=1, inplace=True)

Renaming columns

In [10]:
jrc_plant.rename(columns={'cyear':'year', 'name_p':'name', 'capacity_p': 'capacity_mw',
                      'type_g':'primary_fuel', 'lat': 'latitude', 'lon': 'longitude',
                      'year_commissioned':'commissioning_year', 'Generation': 'generation_mwh',
                      'co2emitted': 'co2_emissions_kg', 'WRI_id': 'gppd_idnr'}, inplace=True)

Unit Conversions: generation_MWh to generation_GWh

In [11]:
jrc_plant['generation_mwh'] = jrc_plant['generation_mwh'].apply(lambda x: x*ureg.MWh)
jrc_plant['generation_mwh'] = jrc_plant['generation_mwh'].apply(lambda x: x.to(ureg.GWh))
jrc_plant['generation_mwh'] = jrc_plant['generation_mwh'].apply(lambda x: x.magnitude)
jrc_plant.rename(columns={'generation_mwh':'generation_gwh'}, inplace=True)

Unit Conversions: co2_emissions_kg to co2_emissions_tonne

In [12]:
jrc_plant['co2_emissions_kg'] = jrc_plant['co2_emissions_kg'].apply(lambda x: x*ureg.kilogram)
jrc_plant['co2_emissions_kg'] = jrc_plant['co2_emissions_kg'].apply(lambda x: x.to(ureg.metric_ton))
jrc_plant['co2_emissions_kg'] = jrc_plant['co2_emissions_kg'].apply(lambda x: x.magnitude)
jrc_plant.rename(columns={'co2_emissions_kg':'co2_emissions_tonne'}, inplace=True)

Changing country name to 3-char ISO code

In [13]:
#country_code= pd.read_csv('https://raw.githubusercontent.com/wri/global-power-plant-database/master/resources/country_information.csv')
#country_code.rename(columns={'primary_country_name':'country'}, inplace=True)

In [14]:
#jrc_plant.loc[jrc_plant['country'] == 'Czechia', 'country'] = 'Czech Republic'
#jrc_plant = pd.merge(jrc_plant,country_code[['country','iso_country_code']],on='country', how='left')
#jrc_plant.drop(['country'], axis=1, inplace=True)
#jrc_plant.rename(columns={'iso_country_code':'country'}, inplace=True)

Changing order of columns in dataframe

In [15]:
jrc_plant= jrc_plant[['name','gppd_idnr','country', 'year',  'co2_emissions_tonne',  
              'capacity_mw','primary_fuel', 'generation_gwh','commissioning_year',
              'latitude', 'longitude', 'unit_count']]  

In [16]:
jrc_plant.head()

,name,gppd_idnr,country,year,co2_emissions_tonne,capacity_mw,primary_fuel,generation_gwh,commissioning_year,latitude,longitude,unit_count
0,KW Hafen,WRI1005901,Germany,2015,1447350.0,300.0,Fossil Hard coal,1621.469,NaN,53.202,8.516,1
1,KW Hafen,WRI1005901,Germany,2016,1369630.0,300.0,Fossil Hard coal,1534.400,NaN,53.202,8.516,1
2,KW Hafen,WRI1005901,Germany,2017,1498930.0,300.0,Fossil Hard coal,1679.258,NaN,53.202,8.516,1
3,KW Hafen,WRI1005901,Germany,2018,1118130.0,300.0,Fossil Hard coal,1252.650,NaN,53.202,8.516,1
4,KW Hastedt,WRI1005903,Germany,2015,781362.0,119.0,Fossil Hard coal,763.137,NaN,53.059,8.873,1


In [17]:
jrc_plant.to_csv('jrc_powerplants.csv', index=False)